<a href="https://colab.research.google.com/github/SHUHI-CHIBA/Behavioral-Recognition/blob/main/%E8%A1%8C%E5%8B%95%E8%AA%8D%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content

! git clone https://github.com/open-mmlab/mmaction2.git

# Commits on Sep 2, 2022使用
%cd /content/mmaction2
!git checkout 3e9e99ff7413b2b5c105586000dc0cc793ce00b5


/content
Cloning into 'mmaction2'...
remote: Enumerating objects: 22864, done.
remote: Total 22864 (delta 0), reused 0 (delta 0), pack-reused 22864
Receiving objects: 100% (22864/22864), 69.62 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (16106/16106), done.
/content/mmaction2
Note: switching to '3e9e99ff7413b2b5c105586000dc0cc793ce00b5'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 3e9e99ff [Doc] support export pdf doc (#1900)


In [ ]:
%cd /content/mmaction2

# For MMAction2
!pip install mmdet==2.25.1
!pip install torch==1.10.1+cu102 torchvision==0.11.2+cu102 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.10/index.html
!pip install -r requirements/build.txt
!pip install -v -e .
!pip install webcolors==1.12

# For clip video
!pip install moviepy==0.2.3.5 imageio==2.4.1
# For download video
!pip install yt-dlp


In [ ]:
!pip install mmcv==1.7.0

In [ ]:
%cd /content/mmaction2

import argparse
import json

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt

import cv2
import torch
from mmaction.apis import init_recognizer, inference_recognizer
from mmcv import Config, DictAction
from mmaction.datasets.pipelines import Compose
from demo.long_video_demo import show_results


In [ ]:
import mmcv
print(mmcv.__version__)

In [ ]:
from mmaction.apis import init_recognizer, inference_recognizer

In [ ]:
%cd /content/mmaction2

# Model Zoo
# https://mmaction2.readthedocs.io/en/latest/modelzoo.html
!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth


In [ ]:
%cd /content/mmaction2

!mkdir input_videos

In [ ]:
import cv2
cap = cv2.VideoCapture("/content/drive/MyDrive/demo2.mp4")
print(cap.isOpened())
cap.release()

In [ ]:
import moviepy.editor

In [ ]:
full_video_path = '/content/drive/MyDrive/demo2.mp4'
output = 'output.mp4'

start_sec =  0#@param {type:"integer"}
end_sec =  3#@param {type:"integer"}
(start_pt, end_pt) = (start_sec, end_sec)


with VideoFileClip('/content/drive/MyDrive/demo2.mp4') as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile('output.mp4')

In [ ]:
clip = moviepy.editor.VideoFileClip("/content/drive/MyDrive/demo2.mp4")
clip = clip.resize(height=420)
clip.ipython_display()


In [ ]:
%cd /content/mmaction2

# Configファイルと学習済みモデルのパスを設定
config_file = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
checkpoint_file = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# 使用デバイスを設定(cuda:0(gpu) or cpu)
device = 'cuda:0'
device = torch.device(device)

# モデルの初期化
model = init_recognizer(config_file, checkpoint_file, device=device)

# 分類ラベルバスを設定
labels = 'tools/data/kinetics/label_map_k400.txt'

# 動画分類
results = inference_recognizer(model, output)

# 分類ラベルをロード
labels = open('tools/data/kinetics/label_map_k400.txt').readlines()
labels = [x.strip() for x in labels]

# 分類結果取得
results = [(labels[k[0]], k[1]) for k in results]

# Top5を表示
print(f'The top-5 labels with corresponding scores are:')
for result in results:
    print(f'{result[0]}: ', result[1])


In [ ]:
%cd /content/mmaction2

device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [ ]:
args = argparse.Namespace(
    config = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py', # configファイルパス
    checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth',      # 学習済みモデルのパス
    video_path = output,                                                             # 入力動画パス
    label = 'tools/data/kinetics/label_map_k400.txt',                                         # 分類ラベル
    out_file = 'demo/result_long_video.mp4',                                                  # 分類結果出力動画
    input_step = 3,                                                                           # フレームをサンプリングするための入力ステップ
    device = device,                                                                          # 使用デバイス cuda:0 or cpu
    threshold = 0.01,                                                                         # スコア閾値
    stride = 0,                                                                               # predict stride = stride * sample_length, 0でpredict stride = 1
    cfg_options = {},                                                                         # configファイルを上書きする場合に設定 key=value
    label_color = (0, 0, 0),                                                            # (B, G, R)出力動画に描画するラベルの文字色
    msg_color = (255, 0, 0),                                                              # (B, G, R)出力動画に描画するメッセージの文字色
)


In [ ]:
# device設定
args.device = torch.device(args.device)
# Configのロード
cfg = Config.fromfile(args.config)
cfg.merge_from_dict(args.cfg_options)
# モデルの初期化
model = init_recognizer(cfg, args.checkpoint, device=args.device)
# データ初期化
data = dict(img_shape=None, modality='RGB', label=-1)
# 分類ラベルロード
with open(args.label, 'r') as f:
  label = [line.strip() for line in f]


In [ ]:
# 予測用パイプライン構築
EXCLUED_STEPS = [
    'OpenCVInit', 'OpenCVDecode', 'DecordInit', 'DecordDecode', 'PyAVInit',
    'PyAVDecode', 'RawFrameDecode'
]

cfg = model.cfg
# sample_length設定
# sample_length分モデルに入力後Action recognition可能
sample_length = 0
pipeline = cfg.data.test.pipeline
pipeline_ = pipeline.copy()
for step in pipeline:
  if 'SampleFrames' in step['type']:
    sample_length = step['clip_len'] * step['num_clips']
    data['num_clips'] = step['num_clips']
    data['clip_len'] = step['clip_len']
    pipeline_.remove(step)
  if step['type'] in EXCLUED_STEPS:
    # デコードフレームステップを除去
    pipeline_.remove(step)
test_pipeline = Compose(pipeline_)


In [ ]:
assert sample_length > 0

args.sample_length = sample_length
args.test_pipeline = test_pipeline

# 予測実行
show_results(model, data, label, args)


In [ ]:
clip = VideoFileClip('demo/result_long_video.mp4')
clip = resize(clip, height=420)
clip.ipython_display()


 99%|█████████▉| 90/91 [00:02<00:00, 30.72it/s]
